In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
import pooch
import os
import scanpy.external as sce
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings ('ignore')

sc.settings.set_figure_params(dpi=600, facecolor="white")

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Arial']
plt.rcParams['font.size'] = 8 

In [2]:
# contamination plot
adata1 = sc.read_h5ad('BCa_mouse_raw.h5ad')
adata1.obs_names_make_unique()
adata1

plt.figure(figsize=(4,4), dpi=300)
plt.hist(adata1.obs['Contamination'], bins=10, color='pink', edgecolor='black', linewidth=0.1)
plt.xlabel('Contamination')
plt.ylabel('Cell Number')
plt.xlim(0.0, 1.0)
plt.xticks(np.arange(0.0, 1.1, 0.2))
plt.title(r'Histogram of Cell Contamination Score')
plt.grid(False)

AnnData object with n_obs × n_vars = 38601 × 29837
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'scDblFinder', 'Contamination'
    layers: 'counts'

In [60]:
# all cells log
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, n_top_genes=3000, batch_key="orig.ident")
sc.tl.pca(adata)

sce.pp.harmony_integrate(adata, 'orig.ident')
adata.obsm['X_pca'] = adata.obsm['X_pca_harmony']

sc.pp.neighbors(adata)
for res in [0.1, 0.3, 0.5, 0.7, 0.9]:
    sc.tl.leiden(
        adata, key_added=f"leiden_res_{res:4.2f}", resolution=res
    )
sc.tl.umap(adata) # umap

2024-08-15 14:50:01,783 - harmonypy - INFO - Iteration 1 of 10
2024-08-15 14:50:06,265 - harmonypy - INFO - Iteration 2 of 10
2024-08-15 14:50:11,030 - harmonypy - INFO - Iteration 3 of 10
2024-08-15 14:50:16,399 - harmonypy - INFO - Iteration 4 of 10
2024-08-15 14:50:21,210 - harmonypy - INFO - Iteration 5 of 10
2024-08-15 14:50:24,636 - harmonypy - INFO - Iteration 6 of 10
2024-08-15 14:50:27,456 - harmonypy - INFO - Iteration 7 of 10
2024-08-15 14:50:29,893 - harmonypy - INFO - Converged after 7 iterations


In [64]:
res=0.4
sc.tl.leiden(adata, key_added=f"leiden_res_{res:4.2f}", resolution=res)
sc.tl.umap(adata) # umap

In [ ]:
with rc_context({'figure.figsize': (3, 3)}):
    sc.pl.umap(adata, color=['leiden_res_0.40','Contamination','CellTypeL0'], ncols = 1)

In [13]:
adata = sc.read_h5ad('BCa_mouse_anno.h5ad')

In [ ]:
marker_genes = {
    'Adipocyte'   : ['Adipoq','Cidec',"Lpl",'Plin4'],
    'Endothelial' : ['Egfl7','Pecam1','Adgrl4','Cdh5'],
    'Epithelial'  : ['Epcam','Krt5','Krt6a','Cldn3'],
    'Fibroblasts' : ['Mmp2','Pdgfra','Lum','Loxl1'],
    'Lymphoid'    : ["Pou2af1", 'Bank1','Jchain','Cd79b'],
    'Myeloid'     : ['C1qb','Lyz2','Hck','Mpeg1'],
    'Neurons'     : ['Syt11','Stmn2','Ndrg4','Kcna6'],
    'Smooth_muscle_cells': ['Myl9','Myh11',"Acta2"]

}
with rc_context({'figure.figsize': (5, 4)}):
    sc.pl.dotplot(adata, marker_genes, groupby="CellTypeL0", standard_scale="var", save='scRNA_BCa_mouse_DotPlot.pdf')

In [5]:
sc.tl.rank_genes_groups(adata, groupby="CellTypeL0", method="wilcoxon")
sc.tl.dendrogram(adata, groupby="CellTypeL0")
# sc.pl.rank_genes_groups_dotplot(
#     adata, groupby="leiden_res_0.40", standard_scale="var", n_genes=10
# )

In [ ]:
sc.get.rank_genes_groups_df(adata, group="Fibroblasts").head(1)

In [68]:
adata
adata.obs['leiden_res_0.40'].value_counts()

adata.obs["Group"] = adata.obs["orig.ident"].map(
    {"R240508001": "6w",
     "R240508002": "6w",
     "R240508003": "8w",
     "R240508004": "8w",
     "R240508005": "12w",
     "R240508006": "12w",
     "R240508007": "24w",
     "R240508008": "24w",
    }
)
adata.write_h5ad('BCa_mouse_anno.h5ad')

leiden_res_0.40
0    8360
1    5801
4    1638
8     641
Name: count, dtype: int64

In [5]:
# Epithelial analysis
epi = adata[adata.obs.CellTypeL0 == 'Epithelial',:]

sc.pp.highly_variable_genes(epi, n_top_genes=3000, batch_key="orig.ident")
sc.tl.pca(epi)

sce.pp.harmony_integrate(epi, 'orig.ident')
epi.obsm['X_pca'] = epi.obsm['X_pca_harmony']

sc.pp.neighbors(adata)
for res in [0.1, 0.3, 0.5, 0.7, 0.9]:
    sc.tl.leiden(
        epi, key_added=f"leiden_res_{res:4.2f}", resolution=res
    )
sc.tl.umap(epi) # umap

epi.write_h5ad('BCa_mouse_epithelial_anno.h5ad')

In [ ]:
adata = sc.read_h5ad('BCa_mouse_anno_L1.h5ad')
with rc_context({'figure.figsize': (3, 3)}):
    sc.pl.umap(adata, color=['CellTypeL1'], ncols = 1)
print(adata.obs['CellTypeL1'].value_counts())

In [2]:
epi = adata[adata.obs.CellTypeL0 == 'Epithelial',:]
sc.pp.highly_variable_genes(epi, n_top_genes=3000, batch_key="orig.ident")
sc.tl.pca(epi)

sce.pp.harmony_integrate(epi, 'orig.ident')
epi.obsm['X_pca'] = epi.obsm['X_pca_harmony']

sc.pp.neighbors(adata)
for res in [0.1, 0.3, 0.5, 0.7, 0.9]:
    sc.tl.leiden(
        epi, key_added=f"leiden_res_{res:4.2f}", resolution=res
    )
sc.tl.umap(epi) # umap

sc.pl.umap(epi, color=['leiden_res_0.10','Contamination','CellTypeL0'], ncols = 1)

epi.write_h5ad('mouse_sc_epi_harmony.h5ad')

,orig.ident,nCount_RNA,nFeature_RNA,scDblFinder,Contamination,leiden_res_0.10,leiden_res_0.30,leiden_res_0.50,leiden_res_0.70,leiden_res_0.90,leiden_res_0.40,CellTypeL0,Group,CellTypeL1
CELL3_N1,R240508004,25822.0,5461,singlet,0.176687,1,1,1,6,8,1,Epithelial,8w,Epi_non_malignant
CELL4_N1,R240508004,1143.0,775,singlet,0.380090,4,5,8,9,10,7,Smooth_muscle_cells,8w,Smooth_muscle_cells
CELL5_N1,R240508004,3672.0,1608,singlet,0.461602,1,1,1,1,3,1,Epithelial,8w,Epi_non_malignant
CELL6_N1,R240508004,4924.0,1802,singlet,0.291693,1,1,1,1,4,1,Epithelial,8w,Epi_non_malignant
CELL9_N1,R240508004,6123.0,1903,singlet,0.197693,1,1,1,1,4,1,Epithelial,8w,Epi_non_malignant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CELL3187_N1-2,R240508006,4271.0,2510,singlet,0.529936,0,0,0,3,1,0,Epithelial,12w,Epi_malignant
CELL3188_N1-3,R240508006,1885.0,1256,singlet,0.542700,0,0,0,3,1,0,Epithelial,12w,Epi_malignant
CELL3189_N1-3,R240508006,2966.0,1553,singlet,0.497286,1,1,1,1,3,1,Epithelial,12w,Epi_non_malignant
CELL3190_N1-2,R240508006,2403.0,1514,singlet,0.107914,0,0,0,0,1,0,Epithelial,12w,Epi_non_malignant


In [55]:
adata = sc.read_h5ad('mouse_sc_epi_harmony.h5ad')

res = "leiden_res_0.10"
sc.tl.rank_genes_groups(adata, groupby=res, method="wilcoxon")
sc.tl.dendrogram(adata, groupby=res,)

sc.pl.rank_genes_groups_dotplot(
    adata, groupby=res, standard_scale="var", n_genes=20
krt20)

In [90]:
sc.get.rank_genes_groups_df(adata, group="1").head(1)

,names,scores,logfoldchanges,pvals,pvals_adj
0,5430419D17Rik,114.806381,3.709452,0.0,0.0


In [ ]:
sc.pl.umap(epi, color=["Pbsn", "Svs2", "Svs4",'Upk1a','Upk3a','Krt17','Upk1b','Upk3a','Upk3b','Cd44','Krt14',
                             'Upk2','Krt18', 'Krt20','Shh','Trp63','Epcam','leiden_res_0.50'], save='Mouse_seminal_vesicle_gland_marker.pdf')

In [ ]:
#sc.pl.umap(adata, color=['leiden_res_0.10'])
#adata.obs['leiden_res_0.10'].value_counts()
marker_genes = {
    'normal' : ['Acer2', 'Agr2', 'Apoc1', 'Cd44', 'Clu', 'Epcam', 'Gata3', 'Hmgb2', 'Id3', 
                'Itga6', 'Klf4', 'Krt14', 'Krt15', 'Krt17', 'Krt18', 'Krt19', 'Krt20', 
                'Krt7', 'Ly6a', 'Mki67', 'Pbsn', 'Pdia6', 'Svs2', 'Svs4', 'Shh', 
                'Tff1', 'Thbs1', 'Top2a', 'Trf', 'Tspan1', 'Ugt2b34', 'Upk1a', 'Upk1b', 
                'Upk2', 'Upk3a', 'Upk3b', 'Upk3bl', 'Ly6a', 'Gsdmc2'],
}
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.10", standard_scale="var")

In [91]:
# Delete seminal vesicles from all cells of mice
epi = sc.read_h5ad('mouse_sc_epi_harmony.h5ad')
cell_ids_to_remove = epi.obs[epi.obs['leiden_res_0.50'].isin(['6', '7'])].index
adata = sc.read_h5ad('mouse_sc_celltypel1.h5ad')
adata = adata[~adata.obs.index.isin(cell_ids_to_remove)].copy()
adata.write_h5ad('mouse_sc_celltypel1_no_gland.h5ad')

In [16]:
# cnv = pd.read_csv('infercnv.observation_groupings.txt', sep=' ')
# cnv = cnv.loc[adata.obs.index]
# cnv
# print(adata.obs.index.equals(cnv.index))
# 
# adata.obs = adata.obs.join(cnv['Dendrogram Group'])
# 
# adata=sc.read_h5ad('mouse_sc_epi_harmony.h5ad')
# 
# epi = adata[adata.obs['Dendrogram Group'] != 'Epithelial_s1',:]
# sc.pp.highly_variable_genes(epi, n_top_genes=3000, batch_key="orig.ident")
# sc.tl.pca(epi)
# 
# sce.pp.harmony_integrate(epi, 'orig.ident')
# epi.obsm['X_pca'] = epi.obsm['X_pca_harmony']
# 
# sc.pp.neighbors(adata)
# for res in [0.1, 0.3, 0.5]:
#     sc.tl.leiden(
#         epi, key_added=f"leiden_res_{res:4.2f}", resolution=res)
# sc.tl.umap(epi) # umap
# 
# epi.obs["cnv_group"] = epi.obs["Dendrogram Group"].map(
#     {"Epithelial_s2": "non_maligant",
#      "Epithelial_s3": "maligant",
#      "Epithelial_s4": "maligant",
#      "Epithelial_s5": "maligant",
#      "Epithelial_s6": "maligant",
#      "Epithelial_s7": "maligant",
#      "Epithelial_s8": "maligant",
#      "Epithelial_s9": "maligant",
#     }
# )
 
# sc.pl.umap(epi, color=['cnv_group','Dendrogram Group'], ncols = 1)
# 
# epi.write_h5ad()
# 
# with rc_context({'figure.figsize': (5, 4)}):
#     sc.pl.umap(epi, color=["Sbp", "Prdx6", "Pbsn", "Nkx3-1", "Msmb", "Spink1",'Krt5','Krt17'])

,Dendrogram Group,Dendrogram Color,Annotation Group,Annotation Color
CELL3_N1,Epithelial_s1,#C6C2C2,1,#8DD3C7
CELL5_N1,Epithelial_s1,#C6C2C2,1,#8DD3C7
CELL6_N1,Epithelial_s1,#C6C2C2,1,#8DD3C7
CELL9_N1,Epithelial_s1,#C6C2C2,1,#8DD3C7
CELL10_N4,Epithelial_s1,#C6C2C2,1,#8DD3C7
...,...,...,...,...
CELL3187_N1-2,Epithelial_s2,#D1C2D2,1,#8DD3C7
CELL3188_N1-3,Epithelial_s2,#D1C2D2,1,#8DD3C7
CELL3189_N1-3,Epithelial_s1,#C6C2C2,1,#8DD3C7
CELL3190_N1-2,Epithelial_s2,#D1C2D2,1,#8DD3C7
